def genre( género : str ): Devuelve el puesto en el que se encuentra un género sobre el ranking de los mismos analizado bajo la columna PlayTimeForever.

In [34]:
import pandas as pd
import numpy as np
import ast
import json

In [24]:
# Leer dataframe de user_items solo con las columnas que se necesitan
columnas = ['item_id','playtime_forever']
df = pd.read_csv('game_items.csv',usecols=columnas)
df.head(3)

,item_id,playtime_forever
0,10,6
1,20,0
2,30,7


In [25]:
# Calcular las horas jugadas para cada videojuego
df_playhours = pd.DataFrame(df.groupby('item_id')['playtime_forever'].agg('sum')).reset_index()
df_playhours.head()

,item_id,playtime_forever
0,10,17386015
1,20,961702
2,30,758991
3,40,154486
4,50,734562


In [26]:
# Leer dataframe de videojuegos solo con las columnas que se necesitan
columnas = ['id','genres']
df = pd.read_csv('steam_games.csv',usecols=columnas)
# renombrar columnas
df.rename(columns={'id':'item_id'},inplace=True)
# Hacer merge (left join) con el df de user_items
df_merge = df_playhours.merge(df,on='item_id',how='left')
# Filtrar por los juegos que tengan playtime mayor a cero 0
df_merge = df_merge[df_merge['playtime_forever'] != 0]
# Eliminar nulos de la columna género
df_merge = df_merge.dropna(subset=['genres'])

In [27]:
df_merge

,item_id,playtime_forever,genres
0,10,17386015,['Action']
1,20,961702,['Action']
2,30,758991,['Action']
3,40,154486,['Action']
4,50,734562,['Action']
...,...,...,...
10967,527510,186,"['Casual', 'Indie', 'Simulation', 'Strategy']"
10970,527810,2,"['Action', 'Indie']"
10971,527890,1,"['Casual', 'Simulation']"
10972,527900,44,"['Adventure', 'Casual']"


In [28]:
# Función para convertir a lista la columna de género
def tolist(lst):
    if isinstance(lst, str) :
        lst = lst.replace("0's",'0s')
        lst =  ast.literal_eval(lst)
        # try:
        #     lst =  ast.literal_eval(lst)
        # except (SyntaxError, ValueError):
        #     return lst.lower()

    return list(lst)

In [29]:
#aplicar la funcion a df_merge para vovler una lista de la columna genero
df_merge['genres'] = df_merge['genres'].apply(lambda x : tolist(x))
df_merge.head()

,item_id,playtime_forever,genres
0,10,17386015,[Action]
1,20,961702,[Action]
2,30,758991,[Action]
3,40,154486,[Action]
4,50,734562,[Action]


In [30]:
# Función para crear las nuevas columnas de género (tipo dummy)
def create_genre_columns(dataframe):
    genres_list = set()
    for genres in dataframe['genres']:
        if isinstance(genres, list):
            genres_list.update(genres)
    
    for genre in genres_list:
        dataframe["genre_"+genre] = dataframe['genres'].apply(lambda x: 1 if genre in x else 0)
    
    return dataframe

In [31]:
#aplicar la funcion a la df_merge
df_with_genre_columns = create_genre_columns(df_merge)
df_with_genre_columns.head()

,item_id,playtime_forever,genres,genre_Racing,genre_Strategy,genre_Indie,genre_Education,genre_Early Access,genre_Software Training,genre_Action,...,genre_Free to Play,genre_Massively Multiplayer,genre_Design &amp; Illustration,genre_Animation &amp; Modeling,genre_Adventure,genre_Photo Editing,genre_RPG,genre_Web Publishing,genre_Simulation,genre_Sports
0,10,17386015,[Action],0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,20,961702,[Action],0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,30,758991,[Action],0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,40,154486,[Action],0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,50,734562,[Action],0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [32]:
genre_columns_mapping = {
    'Racing': 'Racing',
    'Casual': 'Casual',
    'Sports': 'Sports',
    'Indie': 'Indie',
    'Web Publishing': 'Web Publishing',
    'Free to Play': 'Free to Play',
    'Software Training': 'Software Training',
    'RPG': 'RPG',
    'Early Access': 'Early Access',
    'Animation &amp; Modeling': 'Animation &amp; Modeling',
    'Strategy': 'Strategy',
    'Audio Production': 'Audio Production',
    'Action': 'Action',
    'Photo Editing': 'Photo Editing',
    'Massively Multiplayer': 'Massively Multiplayer',
    'Adventure': 'Adventure',
    'Education': 'Education',
    'Video Production': 'Video Production',
    'Simulation': 'Simulation',
    'Utilities': 'Utilities',
    'Design &amp; Illustration': 'Design &amp; Illustration'
}

# Inicializar un diccionario para almacenar las sumas de horas por género
genre_hours_sum = {}

# Iterar a través del diccionario de mapeo de columnas y calcular las sumas de horas
for col_bin, genre in genre_columns_mapping.items():
    if col_bin in df_with_genre_columns.columns:
        genre_hours_sum[genre] = df_with_genre_columns[df_with_genre_columns[col_bin] == 1]['playtime_forever'].sum()

# Crear un DataFrame con las sumas de horas por género
genre_summary_df = pd.DataFrame.from_dict(genre_hours_sum, orient='index', columns=['Total_Hours'])

# Agregar una columna para el ranking de géneros más jugados
genre_summary_df['Rank'] = genre_summary_df['Total_Hours'].rank(ascending=False).astype(int)

# Ordenar el DataFrame por las horas totales en orden descendente
genre_summary_df = genre_summary_df.sort_values(by='Total_Hours', ascending=False)

# Agregar el nombre "Genre" a la primera columna del índice
genre_summary_df = genre_summary_df.rename_axis('Genre')

# Mostrar el DataFrame con el ranking y las sumas de horas por género
genre_summary_df.to_csv('gener_rank.csv')

In [36]:
# Leer el nuevo csv anteriormente creado
df_genre = pd.read_csv('gener_rank.csv')
# Convertir a minúsculas todos los géneros
df_genre['Genre'] = df_genre['Genre'].apply(lambda x: x.lower())
df_genre.to_csv('df-funcion-3.csv',index=False)

In [37]:
# Función para obtener info del género especificado
def get_genre_info(genre):
    genre = genre.lower()
    df_genre = pd.read_csv('df-funcion-3.csv')
    
    if df_genre['Genre'].str.contains(genre).any():
        genre_info = df_genre[df_genre['Genre']==genre]
    else:
        return 'No se encontro el genero'
    return json.dumps(genre_info.to_json(orient='records'), indent=4)
